<a href="https://colab.research.google.com/github/hanzlayyy/FYDP/blob/main/FYDP_Notebook_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [17]:
df = pd.read_csv('/content/sample_data/data v1.csv')

In [18]:
df

,layer_height,infill_density,infill_pattern,nozzle_temperature,print_speed,material,roughness
0,0.02,90.0,grid,220.0,40.0,abs,25
1,0.02,90.0,honeycomb,225.0,40.0,abs,32
2,0.02,80.0,grid,230.0,40.0,abs,40
3,0.02,70.0,honeycomb,240.0,40.0,abs,68
4,0.02,90.0,grid,250.0,40.0,abs,92
...,...,...,...,...,...,...,...
544,0.30,10.0,rectilinear,215.0,120.0,abs,1.64
545,0.30,10.0,rectilinear,215.0,120.0,abs,1.4
546,0.30,10.0,rectilinear,215.0,90.0,abs,0.82
547,0.30,10.0,rectilinear,215.0,90.0,abs,0.76


In [19]:
df=df.drop(403, axis=0)

In [20]:
df['infill_pattern'].unique()

array(['grid', 'honeycomb', 'tetrahedral', nan, 'zigzag', 'triangle',
       'concentric', 'line', 'line ', 'solid', 'rectilinear'],
      dtype=object)

In [21]:
df.isnull().mean()

layer_height          0.000000
infill_density        0.166058
infill_pattern        0.443431
nozzle_temperature    0.000000
print_speed           0.049270
material              0.049270
roughness             0.107664
dtype: float64

In [69]:
df = df.replace('line ', 'line')

In [70]:
missing = df[df.isnull().any(axis=1)]

In [71]:
zero_missing = df[~df.isnull().any(axis=1)]
#zero_missing = pd.get_dummies(zero_missing, columns=['infill_pattern','material'], dtype=int)


In [72]:
one_missing_infill_density = missing[missing['infill_density'].isnull() & ~missing[['infill_pattern','print_speed','material','roughness']].isnull().any(axis=1)]
#one_missing_infill_density = pd.get_dummies(one_missing_infill_density, columns=['infill_pattern','material'], dtype=int)

In [73]:
one_missing_infill_pattern = missing[missing['infill_pattern'].isnull() & ~missing[['infill_density','print_speed','material','roughness']].isnull().any(axis=1)]
#one_missing_infill_pattern = pd.get_dummies(one_missing_infill_pattern, columns=['material'], dtype=int)


In [74]:
one_missing_print_speed = missing[missing['print_speed'].isnull() & ~missing[['infill_density','infill_pattern','material','roughness']].isnull().any(axis=1)]
#one_missing_print_speed = pd.get_dummies(one_missing_print_speed, columns=['infill_pattern','material'], dtype=int)

In [75]:
one_missing_material = missing[missing['material'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','roughness']].isnull().any(axis=1)]
#one_missing_material = pd.get_dummies(one_missing_material, columns=['infill_pattern'], dtype=int)

In [76]:
one_missing_roughness = missing[missing['roughness'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','material']].isnull().any(axis=1)]
#one_missing_roughness = pd.get_dummies(one_missing_roughness, columns=['infill_pattern','material'], dtype=int)

In [78]:
combined = pd.concat([zero_missing, one_missing_infill_density], axis=0)

In [35]:
temp = pd.get_dummies(combined, columns=['infill_pattern','material'], dtype=int)

In [84]:
knn = KNNImputer(n_neighbors=3)
temp2 = pd.DataFrame(knn.fit_transform(temp))
temp2.columns = temp.columns
temp2.isnull().mean().index

Index(['layer_height', 'infill_density', 'nozzle_temperature', 'print_speed',
       'roughness', 'infill_pattern_concentric', 'infill_pattern_grid',
       'infill_pattern_honeycomb', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid',
       'infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag', 'material_abs', 'material_pla',
       'material_pla-aluminum'],
      dtype='object')

In [80]:
zero_missing_1H  = temp2.copy()

In [99]:
C =pd.get_dummies(one_missing_infill_pattern.drop(columns=['infill_pattern']), columns = ['material'],dtype = int)
C['infill_pattern_concentric']= np.NaN
C['material_pla-aluminum'] = 0
D=C.reindex(columns=list(zero_miss_WConc.columns))
D

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [100]:
zero_miss_WConc = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_honeycomb', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_WConc

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,1.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [105]:
knn_1 = KNNImputer(n_neighbors=5)
W = knn_1.fit_transform(zero_miss_WConc)
pd.DataFrame(knn_1.transform(D)).columns = zero_miss_WConc.columns

,0,1,2,3,4,5,6,7,8
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,1.0,0.0
